In [ ]:
%pip install osmnx contextily seaborn

In [ ]:
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import plotly.io as pio

import os
import sys
sys.path.append(os.path.join(os.path.abspath(".."), "functions"))

# Part 1: Estimate the istantaneous traffic 

### Create the project

In [ ]:
PROJECT_NAME = 'AreaVerde'

from environment import dh, pio_renderer
if pio_renderer is not None:
    pio.renderers.default = pio_renderer

project = dh.get_or_create_project(PROJECT_NAME)

### Load the data

In [ ]:
import data_reader
import spatial_utils
import road_network

In [ ]:
gdf_AreaVerde = data_reader.AV_shape(namefile="area_verde_manual_v1.geojson", datapath="data")
buffered_AreaVerde = spatial_utils.buffer_around(gdf=gdf_AreaVerde, buffer_size=3000)

aoi_type = "av"
areas = data_reader.AOI_shapes(namefile="area_verde_manual_v1.geojson", datapath="data", aoi_type=aoi_type, df_around=gdf_AreaVerde)

spira_loc = data_reader.spira_shapes(namefile="SpiraFlowData.parquet", datapath="data", project=project, df_around=gdf_AreaVerde)
spira_ids = data_reader.spira_codes(namefile="SpiraFlowData.parquet", datapath="data", project=project)

In [ ]:
areas

In [ ]:
if os.path.exists("data/results/geo_edges_ALL_v1.geojson") and os.path.exists("data/results/geo_nodes_ALL_v1.geojson"):
    edges, nodes = data_reader.road_data(edges_namefile="geo_edges_ALL_v1.geojson", nodes_namefile="geo_nodes_ALL_v1.geojson", datapath="data/results")
else:
    edges, nodes = road_network.create_road_data(
        connected_network=False, relevant_highway=False,
        edges_namefile="geo_edges_ALL_v1.geojson", nodes_namefile="geo_nodes_ALL_v1.geojson", datapath="data/results",
        df_BAV=buffered_AreaVerde
    )

### Elaborate the data

In [ ]:
import spatial_assignment
import spira_traffic.catchment_area
import data_cleaner

In [ ]:
edges_aoi = spatial_assignment.roads_to_AOI(df_edges=edges, df_aoi=areas)

In [ ]:
spira_close_roads = spatial_assignment.spira_to_road(df_spira=spira_loc, df_edge=edges_aoi)

In [ ]:
# Filter: elements on relevant highways only
spira_close_roads = spira_close_roads[spira_close_roads['highway_ok']]
spira_close_roads = spira_close_roads.reset_index(drop=True).drop(['highway_ok', 'oneway'], axis=1)

edges_aoi_ok = edges_aoi[edges_aoi['highway_ok']]
nodes = nodes[(nodes['node_id'].isin(edges_aoi_ok['u'].values)) | (nodes['node_id'].isin(edges_aoi_ok['v'].values))]

In [ ]:
# Compute the catchment areas of the spiras
if not os.path.exists("data/results/spira-road-time-distances__av_v1.pkl"):
    spira_catchment_area = spira_traffic.catchment_area.find(df_spiras=spira_close_roads, df_edges=edges_aoi_ok, df_nodes=nodes)
    spira_catchment_area.to_pickle("data/results/spira-road-time-distances__av_v1.pkl")
else:
    spira_catchment_area = pd.read_pickle("data/results/spira-road-time-distances__av_v1.pkl")

In [ ]:
# Filter the catchment areas of the spiras
tt = 5.0 * 60 # in seconds
traffic_effect = 0.5 ## To be changed!! 
tt_mod = tt * traffic_effect 

spira_catchment_area = spira_traffic.catchment_area.filter(
    df_catch=spira_catchment_area, 
    time_threshold=tt_mod
)

In [ ]:
# Prepare filters to read spira data
start_date = datetime(2024, 6, 1)
end_date = datetime(2024, 8, 1)
delta_t = 5 #minutes

spira_code_ok = pd.merge(spira_close_roads['spira_unique_id'], spira_ids, how='left')['spira_code'].to_list()

sel = [('start', '>=', start_date.strftime("%Y-%m-%d %H:%M")),
        ('start', '<=', end_date.strftime("%Y-%m-%d %H:%M")), 
        ('sensor_id', 'in', spira_code_ok)]

In [ ]:
# Read and clean spira data
spira5m = data_reader.spira_flows(namefile="SpiraFlowData5m.parquet", datapath="data", project=project, filters=sel)
spira5m = pd.merge(
    spira_ids.rename({'spira_code': 'sensor_id'}, axis=1), 
    spira5m, 
    on='sensor_id', how='inner')
spira5m = data_cleaner.spira_flows(df_spiras=spira5m)

### Compute and save the output

In [ ]:
import spira_traffic.compute_traffic

In [ ]:
traffics = spira_traffic.compute_traffic.per_area(
    spira_data=spira5m, 
    df_catch=spira_catchment_area, 
    first_datetime=start_date, 
    last_datetime=end_date, #start_date+timedelta(minutes=24*60*5),
    deltaTime=delta_t, #30,
    verbose=True
)
traffics.rename(columns={'count_distributed': 'traffic_index'}, inplace=True)
traffics.to_pickle(f"data/results/traffic_5m_estimation__aoi_{aoi_type}.pkl")

In [ ]:
#traffics = spira_traffic.compute_traffic.per_road(
#     spira_data=spira5m, 
#     df_catch=spira_catchment_area, 
#     first_datetime=start_date, 
#     last_datetime=end_date, #start_date+timedelta(minutes=24*60),
#     deltaTime=delta_t, #30
#     verbose=True
# )
# traffics.rename(columns={'count_distributed': 'traffic_index'}, inplace=True)
# traffics.to_pickle(f"data/results/traffic_5m_estimation_roads__aoi_{aoi_type}.pkl")

# Part 2: Compute the mean istantaneous traffic per day

In [ ]:
# Decide whether to use Daytype classification or not
use_daytype = True

# Decide whether to use AOI zoning or not
use_zonegroup = False

### Load the data

In [ ]:
aoi_type = "av"
traffic = pd.read_pickle(f"data/results/traffic_5m_estimation__aoi_{aoi_type}.pkl")

In [ ]:
traffic

### Compute and show output

In [ ]:
import spira_traffic.compute_traffic

In [ ]:
traffic_mean = spira_traffic.compute_traffic.average(
    dataset=traffic, 
    use_daytype=use_daytype, 
    holiday_namefile="data/holiday_list.csv",
    use_zonegroup=use_zonegroup
)
traffic_mean_smooth = spira_traffic.compute_traffic.smoothing(
    dataset=traffic_mean, 
    use_daytype=use_daytype, 
    use_zonegroup=use_zonegroup, 
    method='savgol'
)

In [ ]:
for zone in traffic_mean_smooth['id_zone'].unique():
    data_red = traffic_mean_smooth[traffic_mean_smooth['id_zone']==zone]
    for day in data_red['DayType'].unique():
        data_red_red = data_red[data_red['DayType']==day]
        print(f"array for zone {zone} of day {day}:")
        print(np.array(data_red_red['smooth_traffic']) )

In [ ]:
12 * np.array(
    traffic_mean_smooth
    [traffic_mean_smooth['DayType'] == "Weekday"]
    ['traffic_index']
    .to_list()
)


# Visualization

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.animation import FuncAnimation, PillowWriter
import contextily as ctx

## P1: plot the input and relaborated data

Data input

In [ ]:
# Plot of the AreaVerde area
fig, ax = plt.subplots(figsize=(7,7))
gdf_AreaVerde.boundary.plot(ax=ax, color='green')
gdf_AreaVerde.plot(ax=ax, color='green', edgecolor='green', alpha=0.2)

ctx.add_basemap(ax, crs=gdf_AreaVerde.crs, source=ctx.providers.OpenStreetMap.Mapnik, alpha=0.8)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

In [ ]:
# Plot the road network 
fig, ax = plt.subplots(figsize=(7, 7))
edges_aoi.plot(ax=ax, edgecolor='grey', linewidth=1) ## all roads
edges_aoi_ok.plot(ax=ax, edgecolor='black', linewidth=1) ## important roads
gdf_AreaVerde.plot(ax=ax, color='green', edgecolor='green', alpha=0.2)

ctx.add_basemap(ax, crs=edges_aoi.crs, source=ctx.providers.OpenStreetMap.Mapnik, alpha=0.8)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

In [ ]:
# # Plot the areas of interest
fig, ax = plt.subplots(figsize=(7,7))
areas.plot(ax=ax, column='id_zone', edgecolor='black', cmap='Set2', alpha=0.8)
gdf_AreaVerde.plot(ax=ax, color='none', edgecolor='green')

ctx.add_basemap(ax, crs=areas.crs, source=ctx.providers.OpenStreetMap.Mapnik, alpha=0.8)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

In [ ]:
# Plot the road network colored by the area
fig, ax = plt.subplots(figsize=(7, 7))
gdf_AreaVerde.plot(ax=ax, color='green', alpha=0.2)
areas[areas['id_zone']!='0'].boundary.plot(ax=ax, color='black', alpha=0.8)
edges_aoi[edges_aoi['id_zone']!='0'].plot(ax=ax, column='id_zone', cmap='Set1', linewidth=1)
edges_aoi[edges_aoi['id_zone']=='0'].plot(ax=ax, color='grey', linewidth=1)

ctx.add_basemap(ax, crs=areas.crs, source=ctx.providers.OpenStreetMap.Mapnik, alpha=0.5)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

In [ ]:
# Plot the road network and the spiras
fig, ax = plt.subplots(figsize=(7, 7))
edges_aoi_ok.plot(ax=ax, edgecolor='black', linewidth=1.3)
gdf_AreaVerde.plot(ax=ax, color='green', alpha=0.2)
spira_loc.plot(ax=ax, color='red', marker='o', markersize=10)

ctx.add_basemap(ax, crs=gdf_AreaVerde.crs, source=ctx.providers.OpenStreetMap.Mapnik, alpha=0.8)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

In [ ]:
# Plot of the time distance from the spira 725
id_spira = 3
catch = spira_catchment_area[spira_catchment_area['spira_unique_id']==id_spira]
id_edges_catch = catch['u'] + "_" + catch['v'] + "_" + catch['key']
edges_catch = edges_aoi[edges_aoi['link_id'].isin(id_edges_catch)]

fig, ax = plt.subplots(figsize=(7,7))
edges_catch.plot(ax=ax, edgecolor='red', linewidth=0.5)
gdf_AreaVerde.plot(ax=ax, color='green', edgecolor='green', alpha=0.2)
spira_loc[spira_loc['spira_unique_id'] == id_spira].plot(ax=ax, color='black', marker='o', markersize=25)

ctx.add_basemap(ax, crs=gdf_AreaVerde.crs, source=ctx.providers.OpenStreetMap.Mapnik, alpha=0.8)
plt.xlabel('Longitude')
plt.ylabel('Latitude')

plt.show()

## P2: plot the resulting traffic

### Plot of the time series

In [ ]:
#Plot the time series of the traffic divided by day type
#traffic = traffic[traffic['id_zone'] != '0']
traffic['Date'] = traffic['DateTime'].dt.date
traffic['Time'] = traffic['DateTime'].dt.hour * 3600 + traffic['DateTime'].dt.minute * 60 + traffic['DateTime'].dt.second

if use_daytype:
    plt.figure(figsize=(10, 5))
    sns.lineplot(data=traffic, x='Time', y='traffic_index', hue='DayType', palette={'Weekday': 'blue', 'Saturday': 'green', 'Holiday': 'red'})

    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
    plt.tight_layout(rect=[0, 0, 0.85, 1])

    plt.title('Traffic Index Over Time')
    plt.xlabel('Time (seconds since midnight)')
    plt.ylabel('Traffic Index')

    plt.grid(True)
    plt.show()

In [ ]:
#Plot the time series of the traffic divided by zone id -- only the top-15 zones with highest traffic peak
#traffic = traffic[traffic['id_zone'] != '0']
traffic['Date'] = traffic['DateTime'].dt.date
traffic['Time'] = traffic['DateTime'].dt.hour * 3600 + traffic['DateTime'].dt.minute * 60 + traffic['DateTime'].dt.second

if use_zonegroup:
    top_zones = (
        traffic.groupby('id_zone')['traffic_index']
        .max()
        .nlargest(15)  
        .index
    )
    filtered_traffic = traffic[traffic['id_zone'].isin(top_zones)]

    plt.figure(figsize=(10, 5))
    sns.lineplot(data=filtered_traffic, x='Time', y='traffic_index', hue='id_zone', legend=False, n_boot=250)

    plt.title('Traffic Index Over Time')
    plt.xlabel('Time (seconds since midnight)')
    plt.ylabel('Traffic Index')

    plt.grid(True)
    plt.show()

In [ ]:
# Plot of the traffic flows
traffic_mean = traffic_mean[traffic_mean['id_zone'] != '0']
traffic_mean['Time_s'] = [t.hour * 3600 + t.minute * 60 + t.second for t in traffic_mean['Time'].values]

plt.figure(figsize=(10, 6))
sns.lineplot(data=traffic_mean, x='Time_s', y='traffic_index', hue='id_zone', style='DayType', markers=False, dashes=True, errorbar=None, legend=False)

plt.title('Traffic within the Area(s)')
plt.xlabel('Time')
plt.ylabel('Traffic index')
plt.gca().grid(color='lightgrey', linestyle='-', linewidth=0.5)
plt.show()

In [ ]:
# Plot of the traffic flows when smoothed
traffic_mean_smooth = traffic_mean_smooth[traffic_mean_smooth['id_zone'] != '0']
traffic_mean_smooth['Time_s'] = [t.hour * 3600 + t.minute * 60 + t.second for t in traffic_mean_smooth['Time'].values]

plt.figure(figsize=(10, 6))
sns.lineplot(data=traffic_mean_smooth, x='Time_s', y='smooth_traffic', hue='id_zone', style='DayType', markers=False, dashes=True, legend=False)

plt.title('Smoothed traffic within the Area(s)')
plt.xlabel('Time')
plt.ylabel('Traffic index')
plt.gca().grid(color='lightgrey', linestyle='-', linewidth=0.5)
plt.show()

### Plot of the spatial results

In [ ]:
gdf_AreaVerde = data_reader.AV_shape(namefile="area_verde_manual_v1.geojson", datapath="data")

aoi_type = "od"
areas = data_reader.AOI_shapes(namefile="PROGETTO-AREA.shp", datapath="data", aoi_type=aoi_type, df_around=gdf_AreaVerde)

In [ ]:
#Spatial plot of the results per area
traffics = pd.merge(traffic_mean_smooth, areas).set_geometry('geometry').set_crs(areas.crs)
traffics = traffics[traffics['id_zone'] != '0']
vmin = traffics['smooth_traffic'].min()
vmax = traffics['smooth_traffic'].max()

datetimes = pd.to_datetime(traffics['Time'].apply(lambda t: f"{datetime.today().date()} {t}"))
dates = pd.date_range(start=datetimes.min(), end=datetimes.max(), freq='h').time

def start():
    ax.clear()
    gdf_AreaVerde.plot(ax=ax, color='green', alpha=0.2)
    ctx.add_basemap(ax, crs=areas.crs, source=ctx.providers.OpenStreetMap.Mapnik, alpha=0.8)

def update(frame):
    ax.clear()
    gdf = traffics[(traffics['Time'] >= dates[frame]) & (traffics['Time'] < dates[frame + 1] if frame != (len(dates)-1) else True)]
    gdf = gdf.groupby(['id_zone', 'geometry'])['smooth_traffic'].mean().reset_index().set_geometry('geometry').set_crs(areas.crs)

    if dates[frame] == dates[0]:
        gdf.plot(column='smooth_traffic', ax=ax, legend=True, vmin=vmin, vmax=vmax, cmap='viridis_r')
    else:
        gdf.plot(column='smooth_traffic', ax=ax, legend=False, vmin=vmin, vmax=vmax, cmap='viridis_r')
    ctx.add_basemap(ax, crs=areas.crs, source=ctx.providers.OpenStreetMap.Mapnik, alpha=0.8)
    ax.set_title(f"Traffic Index on {dates[frame]}")

fig, ax = plt.subplots(figsize=(7, 7))
ani = FuncAnimation(fig=fig, func=update, init_func=start, frames=len(dates), repeat=False)
ani.save(f"data/results/traffic_index_area__aoi_{aoi_type}.gif", writer=PillowWriter(fps=2))


In [ ]:
#Spatial plot of the results per area -- weighted by the amount of roads in the area
roads = (
    pd.read_pickle(f'data/results/traffic_5m_estimation_roads__aoi_{aoi_type}.pkl')[['u', 'v', 'key', 'id_zone', 'geometry']]
    .query("id_zone != '0'")
    .drop_duplicates()
    .reset_index(drop=True)
    .pipe(lambda df: df.set_geometry('geometry'))
    .to_crs("EPSG:6875")
    .assign(length= lambda x: x['geometry'].length/1000)
    .groupby(['id_zone'])
    ['length']
    .sum(numeric_only=True)
    .reset_index()
)
areas = areas.merge(roads, how='left')
areas['length'] = areas['length'].fillna(0)

traffics = pd.merge(traffic_mean_smooth, areas).set_geometry('geometry').set_crs(areas.crs)
traffics = traffics[traffics['id_zone'] != '0']
traffics['traffic_index_weighted'] = traffics['traffic_index'] / traffics['length']
traffics.loc[traffics['length'] == 0, 'traffic_index_weighted'] = 0
vmin = traffics['traffic_index_weighted'].min()
vmax = traffics['traffic_index_weighted'].max()

datetimes = pd.to_datetime(traffics['Time'].apply(lambda t: f"{datetime.today().date()} {t}"))
dates = pd.date_range(start=datetimes.min(), end=datetimes.max(), freq='h').time

def start():
    ax.clear()
    gdf_AreaVerde.plot(ax=ax, color='green', alpha=0.2)
    ctx.add_basemap(ax, crs=areas.crs, source=ctx.providers.OpenStreetMap.Mapnik, alpha=0.8)

def update(frame):
    ax.clear()
    gdf = traffics[(traffics['Time'] >= dates[frame]) & (traffics['Time'] < dates[frame + 1] if frame != (len(dates)-1) else True)]
    gdf = gdf.groupby(['id_zone', 'geometry'])['traffic_index_weighted'].mean().reset_index().set_geometry('geometry').set_crs(areas.crs)

    if dates[frame] == dates[0]:
        gdf.plot(column='traffic_index_weighted', ax=ax, legend=True, vmin=vmin, vmax=vmax, cmap='viridis_r')
    else:
        gdf.plot(column='traffic_index_weighted', ax=ax, legend=False, vmin=vmin, vmax=vmax, cmap='viridis_r')
    ctx.add_basemap(ax, crs=areas.crs, source=ctx.providers.OpenStreetMap.Mapnik, alpha=0.8)
    ax.set_title(f"Traffic Index on {dates[frame]}")

fig, ax = plt.subplots(figsize=(7, 7))
ani = FuncAnimation(fig=fig, func=update, init_func=start, frames=len(dates), repeat=False)
ani.save(f"data/results/traffic_index_weighted_area__aoi_{aoi_type}.gif", writer=PillowWriter(fps=2))


### Analysis of smoothing methods

In [ ]:
traffic_mean_smooth['smooth_traffic_savgol'] = traffic_mean_smooth['smooth_traffic']
traffic_mean_smooth = spira_traffic.compute_traffic.smoothing(traffic_mean_smooth, use_daytype=use_daytype, use_zonegroup=use_zonegroup, method='splines')
traffic_mean_smooth.rename({'smooth_traffic': 'smooth_traffic_splines'}, axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(traffic_mean_smooth['Time_s'], traffic_mean_smooth['smooth_traffic_savgol'], label='Sav-Gol Traffic', linestyle='-', color='blue')
plt.plot(traffic_mean_smooth['Time_s'], traffic_mean_smooth['smooth_traffic_splines'], label='Spline Traffic', linestyle='-', color='green')

plt.grid(color='lightgrey', linestyle='-', linewidth=0.5)
plt.title('Traffic Data Visualization')
plt.xlabel('Time')
plt.ylabel('Traffic index')
plt.legend()

plt.show()
